In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install transformers torch torchvision scikit-learn

In [3]:
import os
import pandas as pd
import numpy as np
from PIL import Image, ImageFile
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from torch.optim import AdamW

In [5]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

CSV_PATH = "/content/drive/MyDrive/labels.csv"
IMG_DIR  = "/content/drive/MyDrive/images"

!cp "$CSV_PATH" /content/
!cp -r "$IMG_DIR" /content/


CSV_PATH = "/content/labels.csv"
IMG_DIR  = "/content/images"


df = pd.read_csv(CSV_PATH)

# Sentiment mapping
sentiment_map = {
    "very_negative": 0, "negative": 0,
    "neutral": 1,
    "positive": 2, "very_positive": 2
}
df["sentiment_label"] = df["overall_sentiment"].map(sentiment_map)

# Humor mapping
humor_map = {"not_funny":0, "funny":1, "very_funny":1, "hilarious":1}
df["humor_label"] = df["humour"].map(humor_map)

# Sarcasm mapping
sarcasm_map = {"not_sarcastic":0, "general":1, "twisted_meaning":1, "very_twisted":1}
df["sarcasm_label"] = df["sarcasm"].map(sarcasm_map)

# Offensive mapping
offensive_map = {"not_offensive":0, "slight":1, "very_offensive":1, "hateful_offensive":1}

# Motivational mapping
motivation_map = {"not_motivational":0, "motivational":1}
df["motivation_label"] = df["motivational"].map(motivation_map)


print("Before filtering:", len(df))

# Droping missing images
df = df[df["image_name"].apply(lambda x: os.path.exists(os.path.join(IMG_DIR, x)))]
df = df.reset_index(drop=True)


print("After filtering:", len(df))


# Training

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
print("Train size:", len(train_df), "Val size:", len(val_df))



class MemeDataset(Dataset):
    def __init__(self, df, tokenizer, img_dir, transform=None, max_len=128):
        self.df = df
        self.tokenizer = tokenizer
        self.img_dir = img_dir
        self.transform = transform
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

# Text
        text = str(row["text_corrected"])
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

#  Image
        img_path = os.path.join(self.img_dir, row["image_name"])
        try:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")

            image = torch.zeros(3, 224, 224)


#  Labels

        labels = {
            "sentiment": torch.tensor(row["sentiment_label"]).long(),
            "humor": torch.tensor(row["humor_label"]).long(),
            "sarcasm": torch.tensor(row["sarcasm_label"]).long(),
            "offensive": torch.tensor(row["offensive_label"]).long(),
            "motivation": torch.tensor(row["motivation_label"]).long(),
        }


        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "image": image,
            "labels": labels
        }


# Tokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])



train_dataset = MemeDataset(train_df, tokenizer, IMG_DIR, transform=transform)
val_dataset   = MemeDataset(val_df, tokenizer, IMG_DIR, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16)


# Model

class MultiModalModel(nn.Module):
    def __init__(self):
        super(MultiModalModel, self).__init__()

        # Text Encoder (BERT)
        self.bert = BertModel.from_pretrained("bert-base-uncased")

        # Image Encoder (ResNet18)
        self.cnn = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.cnn.fc = nn.Identity() # Use output before the final FC layer
        self.cnn_fc = nn.Linear(512, 256)



        self.fusion_fc = nn.Linear(self.bert.config.hidden_size + 256, 256)


        self.sentiment_head = nn.Linear(256, 3) # 3 classes for sentiment
        self.humor_head = nn.Linear(256, 2)     # 2 classes for humor
        self.sarcasm_head = nn.Linear(256, 2)   # 2 classes for sarcasm
        self.offensive_head = nn.Linear(256, 2) # 2 classes for offensive
        self.motivation_head = nn.Linear(256, 2)# 2 classes for motivational


    def forward(self, input_ids, attention_mask, images):

        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        text_feat = outputs.pooler_output


        img_feat = self.cnn_fc(self.cnn(images))



        combined = torch.cat((text_feat, img_feat), dim=1)
        fused_features = self.fusion_fc(combined)



        sentiment_logits = self.sentiment_head(fused_features)
        humor_logits = self.humor_head(fused_features)
        sarcasm_logits = self.sarcasm_head(fused_features)
        offensive_logits = self.offensive_head(fused_features)
        motivation_logits = self.motivation_head(fused_features)


        return {
            "sentiment": sentiment_logits,
            "humor": humor_logits,
            "sarcasm": sarcasm_logits,
            "offensive": offensive_logits,
            "motivation": motivation_logits,
        }



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiModalModel().to(device)


criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)


epochs = 2


for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attn_mask = batch["attention_mask"].to(device)
        images    = batch["image"].to(device)
        labels    = batch["labels"]


        labels = {k: v.to(device) for k, v in labels.items()}


        optimizer.zero_grad()
        outputs = model(input_ids, attn_mask, images)


        loss = (
            criterion(outputs["sentiment"], labels["sentiment"]) +
            criterion(outputs["humor"], labels["humor"]) +
            criterion(outputs["sarcasm"], labels["sarcasm"]) +
            criterion(outputs["offensive"], labels["offensive"]) +
            criterion(outputs["motivation"], labels["motivation"])
        )


        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

# Validation
    model.eval()
    correct_sentiment, total_sentiment = 0, 0
    correct_humor, total_humor = 0, 0
    correct_sarcasm, total_sarcasm = 0, 0
    correct_offensive, total_offensive = 0, 0
    correct_motivation, total_motivation = 0, 0


    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attn_mask = batch["attention_mask"].to(device)
            images    = batch["image"].to(device)
            labels    = batch["labels"]


            labels = {k: v.to(device) for k, v in labels.items()}

            outputs = model(input_ids, attn_mask, images)


            _, preds_sentiment = torch.max(outputs["sentiment"], 1)
            correct_sentiment += (preds_sentiment == labels["sentiment"]).sum().item()
            total_sentiment   += labels["sentiment"].size(0)

            _, preds_humor = torch.max(outputs["humor"], 1)
            correct_humor += (preds_humor == labels["humor"]).sum().item()
            total_humor   += labels["humor"].size(0)

            _, preds_sarcasm = torch.max(outputs["sarcasm"], 1)
            correct_sarcasm += (preds_sarcasm == labels["sarcasm"]).sum().item()
            total_sarcasm   += labels["sarcasm"].size(0)

            _, preds_offensive = torch.max(outputs["offensive"], 1)
            correct_offensive += (preds_offensive == labels["offensive"]).sum().item()
            total_offensive   += labels["offensive"].size(0)

            _, preds_motivation = torch.max(outputs["motivation"], 1)
            correct_motivation += (preds_motivation == labels["motivation"]).sum().item()
            total_motivation   += labels["motivation"].size(0)


    val_acc_sentiment = correct_sentiment / total_sentiment
    val_acc_humor = correct_humor / total_humor
    val_acc_sarcasm = correct_sarcasm / total_sarcasm
    val_acc_offensive = correct_offensive / total_offensive
    val_acc_motivation = correct_motivation / total_motivation


    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_loss:.4f} - "
          f"Val Acc (Sentiment): {val_acc_sentiment:.4f} - "
          f"Val Acc (Humor): {val_acc_humor:.4f} - "
          f"Val Acc (Sarcasm): {val_acc_sarcasm:.4f} - "
          f"Val Acc (Offensive): {val_acc_offensive:.4f} - "
          f"Val Acc (Motivation): {val_acc_motivation:.4f}")

Before filtering: 6992
After filtering: 6992
Train size: 5593 Val size: 1399


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 189MB/s]
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/2 - Train Loss: 3.3023 - Val Acc (Sentiment): 0.5904 - Val Acc (Humor): 0.7555 - Val Acc (Sarcasm): 0.7791 - Val Acc (Offensive): 0.5990 - Val Acc (Motivation): 0.6505
Epoch 2/2 - Train Loss: 3.1928 - Val Acc (Sentiment): 0.5890 - Val Acc (Humor): 0.7555 - Val Acc (Sarcasm): 0.7791 - Val Acc (Offensive): 0.6004 - Val Acc (Motivation): 0.6312
